##### Copyright 2025 Google LLC.

In [ ]:
# @title Licenciado bajo la Licencia Apache, Versión 2.0 (la "Licencia");
# no puedes usar este archivo excepto en cumplimiento con la Licencia.
# Puedes obtener una copia de la Licencia en
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# A menos que lo requiera la ley aplicable o se acuerde por escrito, el software
# distribuido bajo la Licencia se distribuye "TAL CUAL",
# SIN GARANTÍAS NI CONDICIONES DE NINGÚN TIPO, ya sean expresas o implícitas.
# Consulta la Licencia para conocer el lenguaje específico que rige los permisos
# y limitaciones bajo la Licencia.

# Día 4 - Ajuste fino de un modelo personalizado

¡Bienvenido de nuevo al curso de Generative AI de 5 días en Kaggle!

En este notebook usarás la API de Gemini para ajustar un modelo personalizado y específico para una tarea. El ajuste fino puede usarse para una variedad de tareas, desde problemas clásicos de PLN como extracción de entidades o resumen, hasta tareas creativas como generación estilizada. Ajustarás un modelo para clasificar el texto (una publicación de un grupo de noticias) en la categoría a la que pertenece (el nombre del grupo de noticias).

Este codelab te guía en el ajuste de un modelo con la API. [AI Studio](https://aistudio.google.com/app/tune) también admite la creación de nuevos modelos ajustados directamente en la interfaz web, lo que te permite crear y monitorear modelos rápidamente usando datos de Google Sheets, Drive o tus propios archivos.

**Nota**: Recomendamos hacer este codelab primero hoy. Puede haber un período de espera mientras el modelo se ajusta, por lo que si comienzas con este, puedes probar el otro codelab mientras esperas.

In [ ]:
!pip uninstall -qqy jupyterlab  # Elimina paquetes conflictivos no utilizados
!pip install -U -q "google-genai==1.7.0"

In [5]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

### Configura tu clave API

Para ejecutar la siguiente celda, tu clave API debe estar almacenada en un [secreto de Kaggle](https://www.kaggle.com/discussions/product-feedback/114053) llamado `GOOGLE_API_KEY`.

Si aún no tienes una clave API, puedes obtener una de [AI Studio](https://aistudio.google.com/app/apikey). Puedes encontrar [instrucciones detalladas en la documentación](https://ai.google.dev/gemini-api/docs/api-key).

Para hacer que la clave esté disponible a través de secretos de Kaggle, elige `Secrets` en el menú `Add-ons` y sigue las instrucciones para agregar tu clave o habilitarla para este notebook.

In [6]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

Si recibiste un error como `No user secrets exist for kernel id ...`, entonces necesitas agregar tu clave API a través de `Add-ons`, `Secrets` **y** habilitarla.

![Captura de pantalla de la casilla para habilitar el secreto GOOGLE_API_KEY](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Explora los modelos disponibles

Usarás el método de la API [`TunedModel.create`](https://ai.google.dev/api/tuning#method:-tunedmodels.create) para iniciar el trabajo de ajuste fino y crear tu modelo personalizado. Encuentra un modelo que lo admita a través del endpoint [`models.list`](https://ai.google.dev/api/models#method:-models.list). También puedes encontrar más información sobre el ajuste de modelos en [la documentación de ajuste de modelos](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?lang=python).

In [7]:
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)

models/gemini-1.5-flash-001-tuning


## Descarga el conjunto de datos

En esta actividad, usarás el mismo conjunto de datos de grupos de noticias que [usaste para entrenar un clasificador en Keras](https://www.kaggle.com/code/markishere/day-2-classifying-embeddings-with-keras/). En este ejemplo, usarás un modelo ajustado de Gemini para lograr el mismo objetivo.

El [Conjunto de Datos de Texto de 20 Grupos de Noticias](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) contiene 18,000 publicaciones de grupos de noticias sobre 20 temas divididos en conjuntos de entrenamiento y prueba.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")

# Ver lista de nombres de clases para el conjunto de datos
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Así es como se ve una sola fila.

In [9]:
print(newsgroups_train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







## Prepara el conjunto de datos

Usarás el mismo código de preprocesamiento que usaste para el modelo personalizado en el día 2. Este preprocesamiento elimina información personal, que puede usarse para "atajar" a usuarios conocidos de un foro, y formatea el texto para que parezca un poco más como texto regular y menos como una publicación de un grupo de noticias (por ejemplo, eliminando los encabezados de correo). Esta normalización permite que el modelo generalice a texto regular y no dependa demasiado de campos específicos. Si tus datos de entrada siempre van a ser publicaciones de grupos de noticias, puede ser útil dejar esta estructura en su lugar si proporcionan señales genuinas.

In [ ]:
import email
import re

import pandas as pd


def preprocess_newsgroup_row(data):
    # Extraer solo el asunto y el cuerpo
    msg = email.message_from_string(data)
    text = f"{msg['Subject']}\n\n{msg.get_payload()}"
    # Eliminar cualquier dirección de correo electrónico restante
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    # Truncar el texto para que se ajuste a los límites de entrada
    text = text[:40000]

    return text


def preprocess_newsgroup_data(newsgroup_dataset):
    # Poner puntos de datos en un dataframe
    df = pd.DataFrame(
        {"Text": newsgroup_dataset.data, "Label": newsgroup_dataset.target}
    )
    # Limpiar el texto
    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)
    # Emparejar la etiqueta con el índice del nombre del objetivo
    df["Class Name"] = df["Label"].map(lambda l: newsgroup_dataset.target_names[l])

    return df

In [ ]:
# Aplicar preprocesamiento a los conjuntos de datos de entrenamiento y prueba
df_train = preprocess_newsgroup_data(newsgroups_train)
df_test = preprocess_newsgroup_data(newsgroups_test)

df_train.head()

,Text,Label,Class Name
0,WHAT car is this!?\n\n I was wondering if anyo...,7,rec.autos
1,SI Clock Poll - Final Call\n\nA fair number of...,4,comp.sys.mac.hardware
2,"PB questions...\n\nwell folks, my mac plus fin...",4,comp.sys.mac.hardware
3,Re: Weitek P9000 ?\n\nRobert J.C. Kyanko () wr...,1,comp.graphics
4,Re: Shuttle Launch Question\n\nFrom article <>...,14,sci.space


Ahora toma una muestra de los datos. Mantendrás 50 filas para cada categoría para el entrenamiento. Ten en cuenta que esto es incluso menos que el ejemplo de Keras, ya que esta técnica (ajuste fino eficiente en parámetros, o PEFT) actualiza un número relativamente pequeño de parámetros y no requiere entrenar un nuevo modelo o actualizar el modelo grande.

In [ ]:
def sample_data(df, num_samples, classes_to_keep):
    # Muestra filas, seleccionando num_samples de cada etiqueta.
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)
    )

    df = df[df["Class Name"].str.contains(classes_to_keep)]
    df["Class Name"] = df["Class Name"].astype("category")

    return df


TRAIN_NUM_SAMPLES = 50
TEST_NUM_SAMPLES = 10
# Mantener rec.* y sci.*
CLASSES_TO_KEEP = "^rec|^sci"

df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)

## Evaluar el rendimiento base

Antes de comenzar a ajustar un modelo, es una buena práctica realizar una evaluación en los modelos disponibles para asegurarte de que puedes medir cuánto ayuda el ajuste.

Primero identifica una sola fila de muestra para usar en la inspección visual.

In [14]:
sample_idx = 0
sample_row = preprocess_newsgroup_row(newsgroups_test.data[sample_idx])
sample_label = newsgroups_test.target_names[newsgroups_test.target[sample_idx]]

print(sample_row)
print('---')
print('Label:', sample_label)

Need info on 88-89 Bonneville


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

---
Label: rec.autos


Pasar el texto directamente como un prompt no produce los resultados deseados. El modelo intentará responder al mensaje.

In [15]:
response = client.models.generate_content(
    model="gemini-1.5-flash-001", contents=sample_row)
print(response.text)

You are right to be confused! Pontiac's Bonneville line in 1988-89 was a bit of a mess. Here's a breakdown:

**Bonneville Models Explained**

* **Base Bonneville:**  The most basic model. Usually had a 3.8L V6 engine and a more basic interior.

* **LE:**  This stood for "Luxury Edition" and typically came with a bit more standard equipment, like upgraded upholstery, power options, and possibly a 5.0L V8 engine. 

* **SE:**  This stood for "Sport Edition." These cars were geared towards a more sporty persona, with features like a firmer suspension, sport wheels, and sometimes a more powerful engine.

* **LSE:**  This is where things get tricky. LSE could stand for "Luxury Sport Edition," which combined features from the LE and SE trims. It also sometimes meant "Limited Sport Edition," which typically had unique styling cues and more limited production.

* **SSE:**  The "SSE" signified "Special Service Edition."  These models had the 5.0L V8 engine and other performance-enhancing feature

Puedes usar las técnicas de ingeniería de prompts que has aprendido esta semana para inducir al modelo a realizar la tarea deseada. Prueba algunas de tus propias ideas y ve qué es efectivo, o revisa las siguientes celdas para diferentes enfoques. ¡Ten en cuenta que tienen diferentes niveles de efectividad!

In [ ]:
# Pregunta al modelo directamente en un prompt de cero disparos.

prompt = "¿De qué grupo de noticias proviene el siguiente mensaje?"
baseline_response = client.models.generate_content(
    model="gemini-1.5-flash-001",
    contents=[prompt, sample_row])
print(baseline_response.text)

The message originates from the **alt.autos.pontiac** newsgroup. 

This is evident from the content of the message, which is specifically focused on a Pontiac model, the Bonneville.  Newsgroup names often reflect the topics discussed within them. 



Esta técnica aún produce una respuesta bastante extensa. Podrías intentar extraer el texto relevante o refinar aún más el prompt.

In [ ]:
from google.api_core import retry

# Puedes usar una instrucción del sistema para hacer un prompt más directo y obtener una
# respuesta más concisa.

system_instruct = """
Eres un servicio de clasificación. Se te pasará una entrada que representa
una publicación de un grupo de noticias y debes responder con el grupo de noticias del que
proviene la publicación.
"""

# Define un ayudante para reintentar cuando se alcance la cuota por minuto.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# Si deseas evaluar tu propia técnica, reemplaza este cuerpo de esta función
# con tu modelo, prompt y otro código y devuelve la respuesta predicha.
@retry.Retry(predicate=is_retriable)
def predict_label(post: str) -> str:
    response = client.models.generate_content(
        model="gemini-1.5-flash-001",
        config=types.GenerateContentConfig(
            system_instruction=system_instruct),
        contents=post)

    rc = response.candidates[0]

    # Cualquier error, filtro, recitación, etc. podemos marcarlo como un error general
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        # Limpiar la respuesta.
        return response.text.strip()


prediction = predict_label(sample_row)

print(prediction)
print()
print("¡Correcto!" si prediction == sample_label else "Incorrecto.")

rec.autos.misc

Incorrect.


Ahora realiza una breve evaluación usando la función definida anteriormente. El conjunto de prueba se muestrea aún más para garantizar que el experimento se ejecute sin problemas en el nivel gratuito de la API. En la práctica, evaluarías todo el conjunto.

In [ ]:
import tqdm
from tqdm.rich import tqdm as tqdmr
import warnings

# Habilitar funciones de tqdm en Pandas.
tqdmr.pandas()

# Pero suprimir la advertencia experimental
warnings.filterwarnings("ignore", category=tqdm.TqdmExperimentalWarning)


# Muestra aún más los datos de prueba para ser consciente del nivel gratuito.
df_baseline_eval = sample_data(df_test, 2, '.*')

# Hacer predicciones usando los datos muestreados.
df_baseline_eval['Prediction'] = df_baseline_eval['Text'].progress_apply(predict_label)

# Y calcular la precisión.
accuracy = (df_baseline_eval["Class Name"] == df_baseline_eval["Prediction"]).sum() / len(df_baseline_eval)
print(f"Precisión: {accuracy:.2%}")

Output()

Accuracy: 37.50%


Ahora echa un vistazo al dataframe para comparar las predicciones con las etiquetas.

In [20]:
df_baseline_eval

,Text,Label,Class Name,Prediction
0,Re: Too fast\n\n (Dan Day) writes:\n> In artic...,7,rec.autos,rec.autos.sports.cars
1,Re: Information needed...\n\n (Youjip Won) wri...,7,rec.autos,comp.sys.ibm.pc.hardware
2,Re: Maxima Chain wax (and mail-order)\n\nIn ar...,8,rec.motorcycles,rec.motorcycles
3,Re: Shaft-drives and Wheelies \n\nIn article <...,8,rec.motorcycles,rec.motorcycles
4,Re: Yanks over A's George Speaks\n\nIn <> wri...,9,rec.sport.baseball,rec.sports.baseball
5,Re: Bosox go down in smoke II (Seattle 7-0) .....,9,rec.sport.baseball,rec.sport.baseball
6,"Selfish hockey fans..\n\n\n\tOn Tuesday, when ...",10,rec.sport.hockey,rec.sports.hockey
7,Re: Octopus in Detroit?\n\nValerie S. Hammerl ...,10,rec.sport.hockey,rec.sport.hockey
8,Re: Let's build software cryptophones for over...,11,sci.crypt,(error)
9,Crypto-PenPals\n\nI came. I lurked. I read the...,11,sci.crypt,sci.crypt


## Ajustar un modelo personalizado

En este ejemplo, usarás el ajuste fino para crear un modelo que no requiera instrucciones del sistema ni prompts y que produzca texto conciso de las clases que proporcionas en los datos de entrenamiento.

Los datos contienen tanto texto de entrada (las publicaciones procesadas) como texto de salida (la categoría, o grupo de noticias), que puedes usar para comenzar a ajustar un modelo.

Al llamar a `tune()`, puedes especificar los hiperparámetros de ajuste del modelo también:
 - `epoch_count`: define cuántas veces recorrer los datos,
 - `batch_size`: define cuántas filas procesar en un solo paso, y
 - `learning_rate`: define el factor de escala para actualizar los pesos del modelo en cada paso.

También puedes optar por omitirlos y usar los valores predeterminados. [Aprende más](https://developers.google.com/machine-learning/crash-course/linear-regression/hyperparameters) sobre estos parámetros y cómo funcionan. Para este ejemplo, estos parámetros se seleccionaron ejecutando algunos trabajos de ajuste y seleccionando parámetros que convergieron eficientemente.

Este ejemplo iniciará un nuevo trabajo de ajuste, pero solo si no existe uno ya. Esto te permite dejar este codelab y volver más tarde: volver a ejecutar este paso encontrará tu último modelo.

In [ ]:
from collections.abc import Iterable
import random


# Convertir el dataframe en un conjunto de datos adecuado para el ajuste.
input_data = {'examples': 
    df_train[['Text', 'Class Name']]
      .rename(columns={'Text': 'textInput', 'Class Name': 'output'})
      .to_dict(orient='records')
 }

# Si estás volviendo a ejecutar este laboratorio, agrega tu model_id aquí.
model_id = None

# O intenta encontrar un trabajo de ajuste reciente.
if not model_id:
  queued_model = None
  # Modelos más nuevos primero.
  for m in reversed(client.tunings.list()):
    # Solo mira los modelos de clasificación de grupos de noticias.
    if m.name.startswith('tunedModels/newsgroup-classification-model'):
      # Si hay un modelo completado, usa el primero (más nuevo).
      if m.state.name == 'JOB_STATE_SUCCEEDED':
        model_id = m.name
        print('Se encontró un modelo ajustado existente para reutilizar.')
        break

      elif m.state.name == 'JOB_STATE_RUNNING' and not queued_model:
        # Si hay un modelo aún en cola, recuerda el más reciente.
        queued_model = m.name
  else:
    if queued_model:
      model_id = queued_model
      print('Se encontró un modelo en cola, aún esperando.')


# Subir los datos de entrenamiento y poner en cola el trabajo de ajuste.
if not model_id:
    tuning_op = client.tunings.tune(
        base_model="models/gemini-1.5-flash-001-tuning",
        training_dataset=input_data,
        config=types.CreateTuningJobConfig(
            tuned_model_display_name="Modelo de clasificación de grupos de noticias",
            batch_size=16,
            epoch_count=2,
        ),
    )

    print(tuning_op.state)
    model_id = tuning_op.name

print(model_id)

/tmp/ipykernel_45/3257069937.py:40: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  tuning_op = client.tunings.tune(


JobState.JOB_STATE_QUEUED
tunedModels/newsgroup-classification-model-yyxc6vlgd


Esto ha creado un trabajo de ajuste que se ejecutará en segundo plano. Para inspeccionar el progreso del trabajo de ajuste, ejecuta esta celda para trazar el estado actual y la curva de pérdida. Una vez que el estado alcance `ACTIVE`, el ajuste estará completo y el modelo estará listo para usar.

Los trabajos de ajuste se ponen en cola, por lo que puede parecer que no se han realizado pasos de entrenamiento inicialmente, pero progresará. El ajuste puede tardar desde unos minutos hasta varias horas, dependiendo de factores como el tamaño de tu conjunto de datos y qué tan ocupada esté la infraestructura de ajuste. ¿Por qué no te tomas una taza de té mientras esperas, o vienes a decir "¡Hola!" en el grupo [Discord](https://discord.com/invite/kaggle).

Es seguro detener esta celda en cualquier momento. No detendrá el trabajo de ajuste.

**IMPORTANTE**: Debido al alto volumen de usuarios que realizan este curso, los trabajos de ajuste pueden estar en cola durante muchas horas. Toma nota de tu ID de modelo ajustado arriba (`tunedModels/...`) para que puedas volver a él mañana. Mientras tanto, revisa el codelab de [Búsqueda de Google](https://www.kaggle.com/code/markishere/day-4-google-search-grounding/). Si deseas intentar ajustar un LLM local, revisa [las guías de ajuste para ajustar un modelo Gemma](https://ai.google.dev/gemma/docs/tune).

In [ ]:
import datetime
import time


MAX_WAIT = datetime.timedelta(minutes=10)

while not (tuned_model := client.tunings.get(name=model_id)).has_ended:

    print(tuned_model.state)
    time.sleep(60)

    # No esperes demasiado. Usa un modelo público si esto va a tardar mucho.
    if datetime.datetime.now(datetime.timezone.utc) - tuned_model.create_time > MAX_WAIT:
        print("Tomando un atajo, usando un modelo preparado previamente.")
        model_id = "tunedModels/newsgroup-classification-model-ltenbi1b"
        tuned_model = client.tunings.get(name=model_id)
        break


print(f"¡Hecho! El estado del modelo es: {tuned_model.state.name}")

if not tuned_model.has_succeeded and tuned_model.error:
    print("Error:", tuned_model.error)

JobState.JOB_STATE_RUNNING
Taking a shortcut, using a previously prepared model.
Done! The model state is: JOB_STATE_SUCCEEDED


## Usa el nuevo modelo

Ahora que tienes un modelo ajustado, pruébalo con datos personalizados. Usas la misma API que una interacción normal de la API de Gemini, pero especificas tu nuevo modelo como el nombre del modelo, que comenzará con `tunedModels/`.

In [ ]:
new_text = """
Primerizo buscando salir de aquí.

Hola, estoy escribiendo sobre mi interés en viajar a los límites exteriores.

¿Qué tipo de nave puedo comprar? ¿Qué es lo más fácil de acceder desde esta tercera roca?

Déjame saber cómo hacer eso, por favor.
"""

response = client.models.generate_content(
    model=model_id, contents=new_text)

print(response.text)

sci.space


### Evaluación

Puedes ver que el modelo produce etiquetas que corresponden a las del conjunto de datos de entrenamiento, y sin ninguna instrucción del sistema o prompt, lo cual ya es una gran mejora. Ahora ve qué tan bien se desempeña en el conjunto de prueba.

Ten en cuenta que no hay paralelismo en este ejemplo; clasificar el subconjunto de prueba tomará unos minutos.

In [ ]:
@retry.Retry(predicate=is_retriable)
def classify_text(text: str) -> str:
    """Clasificar el texto proporcionado en un grupo de noticias conocido."""
    response = client.models.generate_content(
        model=model_id, contents=text)
    rc = response.candidates[0]

    # Cualquier error, filtro, recitación, etc. podemos marcarlo como un error general
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return rc.content.parts[0].text


# El muestreo aquí es solo para minimizar el uso de tu cuota. Si puedes, deberías
# evaluar todo el conjunto de prueba con `df_model_eval = df_test.copy()`.
df_model_eval = sample_data(df_test, 4, '.*')

df_model_eval["Prediction"] = df_model_eval["Text"].progress_apply(classify_text)

accuracy = (df_model_eval["Class Name"] == df_model_eval["Prediction"]).sum() / len(df_model_eval)
print(f"Precisión: {accuracy:.2%}")

Output()

Accuracy: 87.50%


## Comparar el uso de tokens

AI Studio y la API de Gemini proporcionan ajuste de modelos sin costo, sin embargo, se aplican límites y cargos normales para el *uso* de un modelo ajustado.

El tamaño del prompt de entrada y otra configuración de generación como las instrucciones del sistema, así como el número de tokens de salida generados, todos contribuyen al costo general de una solicitud.

In [ ]:
# Calcular el costo de *entrada* del modelo base con instrucciones del sistema.
sysint_tokens = client.models.count_tokens(
    model='gemini-1.5-flash-001', contents=[system_instruct, sample_row]
).total_tokens
print(f'Modelo base con instrucciones del sistema: {sysint_tokens} (entrada)')

# Calcular el costo de entrada del modelo ajustado.
tuned_tokens = client.models.count_tokens(model=tuned_model.base_model, contents=sample_row).total_tokens
print(f'Modelo ajustado: {tuned_tokens} (entrada)')

savings = (sysint_tokens - tuned_tokens) / tuned_tokens
print(f'Ahorro de tokens: {savings:.2%}')  # Ten en cuenta que esto es solo n=1.

System instructed baseline model: 172 (input)
Tuned model: 136 (input)
Token savings: 26.47%


El modelo extenso anterior también produjo más tokens de salida de los necesarios para esta tarea.

In [ ]:
baseline_token_output = baseline_response.usage_metadata.candidates_token_count
print('Tokens de salida del modelo base (extenso):', baseline_token_output)

tuned_model_output = client.models.generate_content(
    model=model_id, contents=sample_row)
tuned_tokens_output = tuned_model_output.usage_metadata.candidates_token_count
print('Tokens de salida del modelo ajustado:', tuned_tokens_output)

Baseline (verbose) output tokens: 52
Tuned output tokens: 4


## Próximos pasos

Ahora que has ajustado un modelo de clasificación, prueba algunas otras tareas, como ajustar un modelo para responder con un tono o estilo específico usando ejemplos escritos a mano (¡o incluso ejemplos generados!). Kaggle alberga [una serie de conjuntos de datos](https://www.kaggle.com/datasets) que puedes probar.

Aprende sobre [cuándo el ajuste fino supervisado es más efectivo](https://cloud.google.com/blog/products/ai-machine-learning/supervised-fine-tuning-for-gemini-llm).

Y revisa el [tutorial de ajuste fino](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?hl=en&lang=python) para otro ejemplo que muestra un modelo ajustado extendiéndose más allá de los datos de entrenamiento a nuevas entradas no vistas.

*- [Mark McD](https://linktr.ee/markmcd)*